# Using FDTD for optical simulations

Finite Differences Time-Domain (FDTD) is a numerical simulation technique for electromagnetic field-propagation. A Yee grid is used, where the magnetic field value in each direction is half a grid point away from the electromagnetic field points. This makes the curl-calculations easy to calculate for both the magnetic field and electric field at each grid point. The electric and magnetic field values of the grid is sequentially updated by the use of Maxwell's equations for each time step in the simulation. 

To be able to use different materials in FDTD, each grid point has a correlated permittivity value and a conductivity value. The permittivity is the real part of the complex permittivity, while the conductivity is calculated by the imaginary part of the complex permittivity, multiplied with the free space permittivity and the angular frequency of the electromagnetic wave. 

In [ ]:
# Append main folder
import sys
sys.path.append("..")

import nidn
import torch

## Single layer TiO2

Configuring the simulation

In [ ]:
cfg = nidn.load_default_cfg()
# Set the number of frequencies to simulate for
cfg.N_freq = 12
#Number of layers with materials
cfg.N_layers = 1
# Define the thickness of each layer
cfg.PER_LAYER_THICKNESS=[1.0]
#Smallest wavelength
cfg.physical_wavelength_range[0]=3e-7
#Largest wavelength
cfg.physical_wavelength_range[1]=1e-6
# Choose FDTD method, TRCWA other option
cfg.solver = "FDTD"
# Set number of time steps in FDTD
cfg.FDTD_niter = 600
# Choose pulse type (continuous, hanning or ricker)
cfg.FDTD_pulse_type = 'continuous'
# Choose source type (line or point)
cfg.FDTD_source_type = 'line'


### Specifiyng the materials

If no material is specified, the grid is assumed to be pure vacuum. By using experimental data, the material properties are calculated for each frequency used in the simulation, by the LayerBuilder

In [ ]:
# Convert wavelengths to normalized frequencies used by the layer builder
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution,
)
# Init layer builder
layer_builder = nidn.LayerBuilder(cfg)
#init epsilon values
eps_grid = torch.zeros(1, 1, cfg.N_layers, cfg.N_freq, dtype=torch.cfloat)
# Set layer to be titanium oxide
eps_grid[:, :, 0, :] = layer_builder.build_uniform_layer("titanium_oxide")


In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
#Plot the spectrum
nidn.plot_spectrum(cfg,R,T)

## Multiple layers

In [ ]:
cfg = nidn.load_default_cfg()
# Set the number of frequencies to simulate for
cfg.N_freq = 12
#Number of layers with materials
cfg.N_layers = 3
# Define the thickness of each layer, in default units
cfg.PER_LAYER_THICKNESS=[1.0, 1.0, 1.0]
#Smallest wavelength
cfg.physical_wavelength_range[0]=3e-7
#Largest wavelength
cfg.physical_wavelength_range[1]=15e-7
# Set number of time steps in FDTD
cfg.FDTD_niter = 2400
# Choose pulse type
cfg.FDTD_pulse_type = 'continuous'

In [ ]:

#init epsilon values
eps_grid = torch.zeros(1, 1, cfg.N_layers, cfg.N_freq, dtype=torch.cfloat)
#Convert wavelengths to normalized frequencies used by the layer builder
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution,
)
# Choose FDTD method, TRCWA other option
cfg.solver = "FDTD"
# Init layer builder
layer_builder = nidn.LayerBuilder(cfg)
# Set the three layers to the desired material
eps_grid[:, :, 0, :] = layer_builder.build_uniform_layer("titanium_oxide")
eps_grid[:, :, 1, :] = layer_builder.build_uniform_layer("gallium_arsenide")
eps_grid[:, :, 2, :] = layer_builder.build_uniform_layer("silicon_nitride")

In [ ]:
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)

In [ ]:
#Plot the spectrum
nidn.plot_spectrum(cfg,R,T)